## Table of Contents

1. Importing Data and Libraries

2. Security Implications: To address any PII data in the data before continuing your analysis.

3. Creating a regional segmentation of the data - “Region” column based on the “State” column from the customer's data set.

4. Creating an exclusion flag for low-activity customers (customers with less than 5 orders) and excluding them from the data.

5. Creating a profiling variable based on age, income, certain goods in the “department_id” column, and number of dependents.

### 1. Importing Data and Libraries

In [1]:
#importing libraries
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns
import scipy
import matplotlib as mpl

In [2]:
# define path
path =r"C:\Users\repla\Documents"

In [3]:
# Import Data
df = pd.read_pickle(r"C:\Users\repla\Documents\02 Data\Prepared Data\orders_products_merged4.9.part1.pkl")

In [4]:
df.head()

,product_id,product_name,aisle_id,department_id,prices,order_id,user_id,order_number,orders_day_of_week,time_of_order,...,mean_expenses,first_name,last_name,gender,state,age,date_joined,n_dependants,fam_status,income
0,1,Chocolate Sandwich Cookies,61,19,5.8,3139998,138,28,6,11,...,5.8,Charles,Cox,Male,Minnesota,81,2019-08-01,1,married,49620
1,1,Chocolate Sandwich Cookies,61,19,5.8,1977647,138,30,6,17,...,5.8,Charles,Cox,Male,Minnesota,81,2019-08-01,1,married,49620
2,907,Premium Sliced Bacon,106,12,20.0,3160996,138,1,5,13,...,20.0,Charles,Cox,Male,Minnesota,81,2019-08-01,1,married,49620
3,907,Premium Sliced Bacon,106,12,20.0,2254091,138,10,5,14,...,20.0,Charles,Cox,Male,Minnesota,81,2019-08-01,1,married,49620
4,1000,Apricots,18,10,12.9,505689,138,9,6,12,...,12.9,Charles,Cox,Male,Minnesota,81,2019-08-01,1,married,49620


In [6]:
# Removing the limit to the amount of columns displayed by default: 
pd.set_option('display.max_columns', None)

### 2. Security Implications

I'll drop the first and last names from the dataset. We can continue with the 'user_id'

In [8]:
# Drop "first_name" and "last_name" column.
df = df.drop(['first_name', 'last_name'], axis=1)

KeyError: "['first_name', 'last_name'] not found in axis"

In [9]:
# I hit this twice by accident, hence the additional error message. Please, disregard.

### 3. Creating a Regional Segementation of the Data

In [10]:
# Creating new regions
Northeast = ['Maine', 'New Hampshire', 'Vermont', 'Massachusetts', 'Rhode Island', 'Connecticut', 'New York', 'Pennsylvania', 'New Jersey']

In [11]:
South = ['Delaware', 'Maryland', 'District of Columbia', 'Virginia', 'West Virginia', 'North Carolina', 'South Carolina', 'Georgia', 'Florida', 'Kentucky', 'Tennessee', 'Mississippi', 'Alabama', 'Oklahoma', 'Texas', 'Arkansas', 'Louisiana']

In [12]:
Midwest = ['Wisconsin', 'Michigan', 'Illinois', 'Indiana', 'Ohio', 'North Dakota', 'South Dakota', 'Nebraska', 'Kansas', 'Minnesota', 'Iowa', 'Missouri']

In [13]:
West = ['Idaho', 'Montana', 'Wyoming', 'Nevada', 'Utah', 'Colorado', 'Arizona', 'New Mexico', 'Alaska', 'Washington', 'Oregon', 'California', 'Hawaii']

In [14]:
# Creating appropriate columns with our new Regions
df.loc[df['state'].isin(Northeast), 'region'] = 'Northeast'

C:\Users\repla\AppData\Local\Temp\ipykernel_9232\1537082384.py:2: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'Northeast' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.loc[df['state'].isin(Northeast), 'region'] = 'Northeast'


In [15]:
df.loc[df['state'].isin(Midwest), 'region'] = 'Midwest'

In [16]:
df.loc[df['state'].isin(South), 'region'] = 'South'

In [17]:
df.loc[df['state'].isin(West), 'region'] = 'West'

In [18]:
# Confirming that the new region columns are successfully created 
df['region'].value_counts(dropna = False)

region
South        333337
West         255990
Midwest      233768
Northeast    176905
Name: count, dtype: int64

In [19]:
df.head()

,product_id,product_name,aisle_id,department_id,prices,order_id,user_id,order_number,orders_day_of_week,time_of_order,days_since_prior_order,add_to_cart_order,reordered,price_range,price_range_loc,busiest_day,busiest_days,busiest_period_of_day,max_order,loyalty_flag,average_price,spending_flag,median_orders,order_frequency_flag,mean_expenses,gender,state,age,date_joined,n_dependants,fam_status,income,region
0,1,Chocolate Sandwich Cookies,61,19,5.8,3139998,138,28,6,11,3.0,5,0,Mid-range product,Mid-range product,Regularly busy,Regularly busy,Most orders,30,Regular customer,12.9,High spender,6.5,Frequent customer,5.8,Male,Minnesota,81,2019-08-01,1,married,49620,Midwest
1,1,Chocolate Sandwich Cookies,61,19,5.8,1977647,138,30,6,17,20.0,1,1,Mid-range product,Mid-range product,Regularly busy,Regularly busy,Average orders,30,Regular customer,12.9,High spender,6.5,Frequent customer,5.8,Male,Minnesota,81,2019-08-01,1,married,49620,Midwest
2,907,Premium Sliced Bacon,106,12,20.0,3160996,138,1,5,13,NaN,4,0,High range,High-range product,Regularly busy,Regularly busy,Most orders,30,Regular customer,12.9,High spender,6.5,Frequent customer,20.0,Male,Minnesota,81,2019-08-01,1,married,49620,Midwest
3,907,Premium Sliced Bacon,106,12,20.0,2254091,138,10,5,14,6.0,1,1,High range,High-range product,Regularly busy,Regularly busy,Most orders,30,Regular customer,12.9,High spender,6.5,Frequent customer,20.0,Male,Minnesota,81,2019-08-01,1,married,49620,Midwest
4,1000,Apricots,18,10,12.9,505689,138,9,6,12,7.0,5,0,Mid-range product,Mid-range product,Regularly busy,Regularly busy,Most orders,30,Regular customer,12.9,High spender,6.5,Frequent customer,12.9,Male,Minnesota,81,2019-08-01,1,married,49620,Midwest


In [20]:
# Crosstab - crossing the variable 'regions' with the 'spending_flag' variable
crosstab = pd.crosstab(df['spending_flag'], df['region'], dropna = False)

In [23]:
crosstab.to_clipboard()

In [22]:
crosstab

region,Midwest,Northeast,South,West
spending_flag,,,,
High spender,45824,34605,64044,50021
Low spender,187944,142300,269293,205969


In [24]:
# I was surprised to discover that our spender percentage was incredibly similar throughout the 4 Regions. The Midwest and Northeast both had 24.3% of the population as high spenders, while the West was just behind with 24.2&, and the South at 23.7%

### 4. Creating an Exclusion Flag for Low-Activity Customers

In [25]:
df.loc[df['max_order'] < 5, 'activity_flag'] = 'low activity'

C:\Users\repla\AppData\Local\Temp\ipykernel_9232\2189153115.py:1: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'low activity' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.loc[df['max_order'] < 5, 'activity_flag'] = 'low activity'


In [26]:
df.loc[df['max_order'] >= 5, 'activity_flag'] = 'active customer'

In [27]:
df['activity_flag'].value_counts(dropna=False)

activity_flag
active customer    921270
low activity        78730
Name: count, dtype: int64

In [28]:
# creat new subset of low-activity customers
df_low_activity = df[df['activity_flag'] =='low activity']

In [29]:
df_active_customers = df[df['activity_flag'] =='active customer']

In [30]:
df_low_activity

,product_id,product_name,aisle_id,department_id,prices,order_id,user_id,order_number,orders_day_of_week,time_of_order,days_since_prior_order,add_to_cart_order,reordered,price_range,price_range_loc,busiest_day,busiest_days,busiest_period_of_day,max_order,loyalty_flag,average_price,spending_flag,median_orders,order_frequency_flag,mean_expenses,gender,state,age,date_joined,n_dependants,fam_status,income,region,activity_flag
14,1,Chocolate Sandwich Cookies,61,19,5.8,652770,764,1,3,13,NaN,10,0,Mid-range product,Mid-range product,Regularly busy,Least busy days,Most orders,3,New customer,5.8,Low spender,9.0,Frequent customer,5.8,Female,Wisconsin,40,2020-02-09,3,married,31308,Midwest,low activity
15,1,Chocolate Sandwich Cookies,61,19,5.8,1813452,764,3,4,17,9.0,11,1,Mid-range product,Mid-range product,Least busy,Least busy days,Average orders,3,New customer,5.8,Low spender,9.0,Frequent customer,5.8,Female,Wisconsin,40,2020-02-09,3,married,31308,Midwest,low activity
17,1,Chocolate Sandwich Cookies,61,19,5.8,1871483,825,3,2,14,30.0,2,0,Mid-range product,Mid-range product,Regularly busy,Regularly busy,Most orders,3,New customer,5.8,Low spender,30.0,Non-frequent customer,5.8,Male,District of Columbia,20,2019-05-08,3,living with parents and siblings,34171,South,low activity
94,1,Chocolate Sandwich Cookies,61,19,5.8,2434812,1598,1,2,12,NaN,2,0,Mid-range product,Mid-range product,Regularly busy,Regularly busy,Most orders,3,New customer,6.6,Low spender,2.0,Frequent customer,5.8,Male,Kansas,41,2018-01-07,2,married,122060,Midwest,low activity
95,1,Chocolate Sandwich Cookies,61,19,5.8,1531889,1598,2,2,10,21.0,2,1,Mid-range product,Mid-range product,Regularly busy,Regularly busy,Most orders,3,New customer,6.6,Low spender,2.0,Frequent customer,5.8,Male,Kansas,41,2018-01-07,2,married,122060,Midwest,low activity
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
999979,2078,Unsalted Pure Irish Butter,36,16,13.5,242719,99886,1,6,11,NaN,16,0,Mid-range product,Mid-range product,Regularly busy,Regularly busy,Most orders,1,New customer,13.5,High spender,NaN,NaN,13.5,Female,North Carolina,29,2018-01-25,1,married,87293,South,low activity
999983,2078,Unsalted Pure Irish Butter,36,16,13.5,625446,100309,2,3,9,30.0,7,0,Mid-range product,Mid-range product,Regularly busy,Least busy days,Most orders,2,New customer,13.5,High spender,30.0,Non-frequent customer,13.5,Male,Tennessee,28,2019-08-15,0,single,32030,South,low activity
999997,2078,Unsalted Pure Irish Butter,36,16,13.5,3310791,102530,1,4,17,NaN,13,0,Mid-range product,Mid-range product,Least busy,Least busy days,Average orders,3,New customer,13.5,High spender,17.0,Regular customer,13.5,Female,Maine,48,2018-04-06,1,married,136558,Northeast,low activity
999998,2078,Unsalted Pure Irish Butter,36,16,13.5,1704867,102530,2,2,17,30.0,4,1,Mid-range product,Mid-range product,Regularly busy,Regularly busy,Average orders,3,New customer,13.5,High spender,17.0,Regular customer,13.5,Female,Maine,48,2018-04-06,1,married,136558,Northeast,low activity


In [31]:
df_active_customers

,product_id,product_name,aisle_id,department_id,prices,order_id,user_id,order_number,orders_day_of_week,time_of_order,days_since_prior_order,add_to_cart_order,reordered,price_range,price_range_loc,busiest_day,busiest_days,busiest_period_of_day,max_order,loyalty_flag,average_price,spending_flag,median_orders,order_frequency_flag,mean_expenses,gender,state,age,date_joined,n_dependants,fam_status,income,region,activity_flag
0,1,Chocolate Sandwich Cookies,61,19,5.8,3139998,138,28,6,11,3.0,5,0,Mid-range product,Mid-range product,Regularly busy,Regularly busy,Most orders,30,Regular customer,12.9,High spender,6.5,Frequent customer,5.8,Male,Minnesota,81,2019-08-01,1,married,49620,Midwest,active customer
1,1,Chocolate Sandwich Cookies,61,19,5.8,1977647,138,30,6,17,20.0,1,1,Mid-range product,Mid-range product,Regularly busy,Regularly busy,Average orders,30,Regular customer,12.9,High spender,6.5,Frequent customer,5.8,Male,Minnesota,81,2019-08-01,1,married,49620,Midwest,active customer
2,907,Premium Sliced Bacon,106,12,20.0,3160996,138,1,5,13,NaN,4,0,High range,High-range product,Regularly busy,Regularly busy,Most orders,30,Regular customer,12.9,High spender,6.5,Frequent customer,20.0,Male,Minnesota,81,2019-08-01,1,married,49620,Midwest,active customer
3,907,Premium Sliced Bacon,106,12,20.0,2254091,138,10,5,14,6.0,1,1,High range,High-range product,Regularly busy,Regularly busy,Most orders,30,Regular customer,12.9,High spender,6.5,Frequent customer,20.0,Male,Minnesota,81,2019-08-01,1,married,49620,Midwest,active customer
4,1000,Apricots,18,10,12.9,505689,138,9,6,12,7.0,5,0,Mid-range product,Mid-range product,Regularly busy,Regularly busy,Most orders,30,Regular customer,12.9,High spender,6.5,Frequent customer,12.9,Male,Minnesota,81,2019-08-01,1,married,49620,Midwest,active customer
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
999992,2078,Unsalted Pure Irish Butter,36,16,13.5,569026,101704,5,1,17,30.0,3,1,Mid-range product,Mid-range product,Regularly busy,Busiest days,Average orders,12,Regular customer,13.5,High spender,21.0,Non-frequent customer,13.5,Female,Florida,25,2017-01-22,3,married,62540,South,active customer
999993,2078,Unsalted Pure Irish Butter,36,16,13.5,3217584,101704,6,1,19,21.0,4,1,Mid-range product,Mid-range product,Regularly busy,Busiest days,Average orders,12,Regular customer,13.5,High spender,21.0,Non-frequent customer,13.5,Female,Florida,25,2017-01-22,3,married,62540,South,active customer
999994,2078,Unsalted Pure Irish Butter,36,16,13.5,630319,101704,8,1,14,29.0,5,1,Mid-range product,Mid-range product,Regularly busy,Busiest days,Most orders,12,Regular customer,13.5,High spender,21.0,Non-frequent customer,13.5,Female,Florida,25,2017-01-22,3,married,62540,South,active customer
999995,2078,Unsalted Pure Irish Butter,36,16,13.5,2712923,101704,10,1,8,21.0,9,1,Mid-range product,Mid-range product,Regularly busy,Busiest days,Average orders,12,Regular customer,13.5,High spender,21.0,Non-frequent customer,13.5,Female,Florida,25,2017-01-22,3,married,62540,South,active customer


In [32]:
# Saving the low activity customers as a pickle. 
df_low_activity.to_pickle(os.path.join(path, '02 Data','Prepared Data', 'df_low_activity_cutomers.pkl'))

In [33]:
df_active_customers.to_pickle(os.path.join(path, '02 Data','Prepared Data', 'df_active_cutomers.pkl'))

### 5. Creating a Profiling Variable 
Age, Income, Certain goods in the 'department_id', and Number of Dependants

In [34]:
# Age
df_active_customers['age'].describe()

count    921270.000000
mean         49.484424
std          18.508229
min          18.000000
25%          33.000000
50%          49.000000
75%          65.000000
max          81.000000
Name: age, dtype: float64

In [38]:
# Create age groups
df_active_customers.loc[(df_active_customers['age'] >= 18) & (df_active_customers['age'] <= 29), 'age_group'] = '18-29 years old' 

In [37]:
df_active_customers.loc[(df_active_customers['age'] >= 30) & (df_active_customers['age'] <= 44), 'age_group'] = '30-44 years old'

In [39]:
df_active_customers.loc[(df_active_customers['age'] >= 45) & (df_active_customers['age'] <= 59), 'age_group'] = '45-59 years old'


In [40]:
df_active_customers.loc[(df_active_customers['age'] >= 60) & (df_active_customers['age'] <= 81), 'age_group'] = '60-81 years old'

In [41]:
df_active_customers['age_group'].value_counts(dropna = False)

age_group
60-81 years old    314214
45-59 years old    219449
30-44 years old    213714
18-29 years old    173893
Name: count, dtype: int64

In [42]:
# Income
df_active_customers['income'].describe()

count    921270.000000
mean      99989.377622
std       44934.372047
min       25911.000000
25%       66981.000000
50%       96753.000000
75%      128375.000000
max      593901.000000
Name: income, dtype: float64

In [ ]:
# Creating income groups = I used <66981 as Low income, >66981 - <96753 as Middle income, and >96753 as High Income


In [43]:
df_active_customers.loc[df_active_customers['income'] <= 66981, 'income_group'] = 'Low Income'

C:\Users\repla\AppData\Local\Temp\ipykernel_9232\1311929826.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_active_customers.loc[df_active_customers['income'] <= 66981, 'income_group'] = 'Low Income'
C:\Users\repla\AppData\Local\Temp\ipykernel_9232\1311929826.py:1: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'Low Income' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df_active_customers.loc[df_active_customers['income'] <= 66981, 'income_group'] = 'Low Income'


In [44]:
df_active_customers.loc[(df_active_customers['income'] > 66981) & (df_active_customers['income'] <= 96753), 'income_group'] = 'Middle Income'

In [45]:
df_active_customers.loc[df_active_customers['income'] > 96753, 'income_group'] = 'High Income'

In [46]:
df_active_customers.head()

,product_id,product_name,aisle_id,department_id,prices,order_id,user_id,order_number,orders_day_of_week,time_of_order,days_since_prior_order,add_to_cart_order,reordered,price_range,price_range_loc,busiest_day,busiest_days,busiest_period_of_day,max_order,loyalty_flag,average_price,spending_flag,median_orders,order_frequency_flag,mean_expenses,gender,state,age,date_joined,n_dependants,fam_status,income,region,activity_flag,age_group,income_group
0,1,Chocolate Sandwich Cookies,61,19,5.8,3139998,138,28,6,11,3.0,5,0,Mid-range product,Mid-range product,Regularly busy,Regularly busy,Most orders,30,Regular customer,12.9,High spender,6.5,Frequent customer,5.8,Male,Minnesota,81,2019-08-01,1,married,49620,Midwest,active customer,60-81 years old,Low Income
1,1,Chocolate Sandwich Cookies,61,19,5.8,1977647,138,30,6,17,20.0,1,1,Mid-range product,Mid-range product,Regularly busy,Regularly busy,Average orders,30,Regular customer,12.9,High spender,6.5,Frequent customer,5.8,Male,Minnesota,81,2019-08-01,1,married,49620,Midwest,active customer,60-81 years old,Low Income
2,907,Premium Sliced Bacon,106,12,20.0,3160996,138,1,5,13,NaN,4,0,High range,High-range product,Regularly busy,Regularly busy,Most orders,30,Regular customer,12.9,High spender,6.5,Frequent customer,20.0,Male,Minnesota,81,2019-08-01,1,married,49620,Midwest,active customer,60-81 years old,Low Income
3,907,Premium Sliced Bacon,106,12,20.0,2254091,138,10,5,14,6.0,1,1,High range,High-range product,Regularly busy,Regularly busy,Most orders,30,Regular customer,12.9,High spender,6.5,Frequent customer,20.0,Male,Minnesota,81,2019-08-01,1,married,49620,Midwest,active customer,60-81 years old,Low Income
4,1000,Apricots,18,10,12.9,505689,138,9,6,12,7.0,5,0,Mid-range product,Mid-range product,Regularly busy,Regularly busy,Most orders,30,Regular customer,12.9,High spender,6.5,Frequent customer,12.9,Male,Minnesota,81,2019-08-01,1,married,49620,Midwest,active customer,60-81 years old,Low Income


In [47]:
df_active_customers['income_group'].value_counts(dropna=False)

income_group
High Income      460633
Low Income       230344
Middle Income    230293
Name: count, dtype: int64

In [49]:
# Download department_id, create labels, and merge data
df_department = pd.read_csv(os.path.join(path,'02 Data', 'Prepared Data', 'departments_wrangled.csv'))

In [50]:
df_department

,Unnamed: 0,department
0,1,frozen
1,2,other
2,3,bakery
3,4,produce
4,5,alcohol
5,6,international
6,7,beverages
7,8,pets
8,9,dry goods pasta
9,10,bulk


In [51]:
df_department.rename(columns = {'Unnamed: 0' : 'department_id'}, inplace = True)

In [52]:
df_department['department_id'].describe()

count    21.000000
mean     11.000000
std       6.204837
min       1.000000
25%       6.000000
50%      11.000000
75%      16.000000
max      21.000000
Name: department_id, dtype: float64

In [53]:
df_department

,department_id,department
0,1,frozen
1,2,other
2,3,bakery
3,4,produce
4,5,alcohol
5,6,international
6,7,beverages
7,8,pets
8,9,dry goods pasta
9,10,bulk


In [54]:
# Completing the merge
df_active_customers = df_active_customers.merge(df_department, on = 'department_id')

In [55]:
df_active_customers.head()

,product_id,product_name,aisle_id,department_id,prices,order_id,user_id,order_number,orders_day_of_week,time_of_order,days_since_prior_order,add_to_cart_order,reordered,price_range,price_range_loc,busiest_day,busiest_days,busiest_period_of_day,max_order,loyalty_flag,average_price,spending_flag,median_orders,order_frequency_flag,mean_expenses,gender,state,age,date_joined,n_dependants,fam_status,income,region,activity_flag,age_group,income_group,department
0,1,Chocolate Sandwich Cookies,61,19,5.8,3139998,138,28,6,11,3.0,5,0,Mid-range product,Mid-range product,Regularly busy,Regularly busy,Most orders,30,Regular customer,12.900000,High spender,6.5,Frequent customer,5.8,Male,Minnesota,81,2019-08-01,1,married,49620,Midwest,active customer,60-81 years old,Low Income,snacks
1,1,Chocolate Sandwich Cookies,61,19,5.8,1977647,138,30,6,17,20.0,1,1,Mid-range product,Mid-range product,Regularly busy,Regularly busy,Average orders,30,Regular customer,12.900000,High spender,6.5,Frequent customer,5.8,Male,Minnesota,81,2019-08-01,1,married,49620,Midwest,active customer,60-81 years old,Low Income,snacks
2,1,Chocolate Sandwich Cookies,61,19,5.8,389851,709,2,0,21,6.0,20,0,Mid-range product,Mid-range product,Busiest day,Busiest days,Average orders,5,New customer,5.288889,Low spender,7.0,Frequent customer,5.8,Female,Vermont,66,2018-06-16,2,married,158302,Northeast,active customer,60-81 years old,High Income,snacks
3,955,Cheddar Chex Mix,125,19,3.9,147387,709,4,0,21,8.0,30,0,Low-range product,Low-range product,Busiest day,Busiest days,Average orders,5,New customer,5.288889,Low spender,7.0,Frequent customer,3.9,Female,Vermont,66,2018-06-16,2,married,158302,Northeast,active customer,60-81 years old,High Income,snacks
4,1,Chocolate Sandwich Cookies,61,19,5.8,1701441,777,16,1,7,26.0,7,0,Mid-range product,Mid-range product,Regularly busy,Busiest days,Average orders,16,Regular customer,5.800000,Low spender,26.0,Non-frequent customer,5.8,Female,Hawaii,51,2019-01-04,2,married,57797,West,active customer,45-59 years old,Low Income,snacks


In [56]:
# Creat dependants group in dataset
df_active_customers.loc[df_active_customers['n_dependants'] == 0, 'dependants_group'] = 'No dependants'

C:\Users\repla\AppData\Local\Temp\ipykernel_9232\4072779817.py:2: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'No dependants' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df_active_customers.loc[df_active_customers['n_dependants'] == 0, 'dependants_group'] = 'No dependants'


In [57]:
df_active_customers.loc[df_active_customers['n_dependants'] > 0, 'dependants_group'] = 'Has dependants'

In [59]:
df_active_customers['n_dependants'].value_counts()

n_dependants
0    231318
2    230785
3    230669
1    228498
Name: count, dtype: int64

In [61]:
#Confirming our group number adds up to 921270
df_active_customers['dependants_group'].value_counts()

dependants_group
Has dependants    689952
No dependants     231318
Name: count, dtype: int64

In [63]:
#Creating family profile group in dataset
df_active_customers['fam_status'].value_counts(dropna = False)

fam_status
married                             645202
single                              152701
divorced/widowed                     78617
living with parents and siblings     44750
Name: count, dtype: int64

In [64]:
df_active_customers['gender'].value_counts(dropna = False)

gender
Male      463221
Female    458049
Name: count, dtype: int64

In [65]:
#Creating a 'family_profile' column in the dataset
df_active_customers.loc[(df_active_customers['fam_status'] == 'married') & (df_active_customers['dependants_group'] == 'No dependants') & (df_active_customers['gender'] == 'Female'), 'family_profile'] = 'married female, no children'

C:\Users\repla\AppData\Local\Temp\ipykernel_9232\468441552.py:2: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'married female, no children' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df_active_customers.loc[(df_active_customers['fam_status'] == 'married') & (df_active_customers['dependants_group'] == 'No dependants') & (df_active_customers['gender'] == 'Female'), 'family_profile'] = 'married female, no children'


In [66]:
df_active_customers.loc[(df_active_customers['fam_status'] == 'married') & (df_active_customers['dependants_group'] == 'No dependants') & (df_active_customers['gender'] == 'Male'), 'family_profile'] = 'married male, no children'

In [67]:
df_active_customers.loc[(df_active_customers['fam_status'] == 'married') & (df_active_customers['dependants_group'] == 'Has dependants') & (df_active_customers['gender'] == 'Female'), 'family_profile'] = 'married female with children'

In [68]:
df_active_customers.loc[(df_active_customers['fam_status'] == 'married') & (df_active_customers['dependants_group'] == 'Has dependants') & (df_active_customers['gender'] == 'Male'), 'family_profile'] = 'married male with children'

In [69]:
df_active_customers.loc[(df_active_customers['fam_status'].isin(['single','living with parents and siblings','divorced/widowed'])) & (df_active_customers['dependants_group'] == 'No dependants') & (df_active_customers['gender'] == 'Female'), 'family_profile'] = 'single female, no children'

In [70]:
df_active_customers.loc[(df_active_customers['fam_status'].isin(['single','living with parents and siblings','divorced/widowed'])) & (df_active_customers['dependants_group'] == 'No dependants') & (df_active_customers['gender'] == 'Male'), 'family_profile'] = 'single male, no children'

In [71]:
df_active_customers.loc[(df_active_customers['fam_status'].isin(['single','living with parents and siblings','divorced/widowed'])) & (df_active_customers['dependants_group'] == 'Has dependants') & (df_active_customers['gender'] == 'Female'), 'family_profile'] = 'single female with children'

In [72]:
df_active_customers.loc[(df_active_customers['fam_status'].isin(['single','living with parents and siblings','divorced/widowed'])) & (df_active_customers['dependants_group'] == 'Has dependants') & (df_active_customers['gender'] == 'Male'), 'family_profile'] = 'single male with children'

In [73]:
# Checking my work above
df_active_customers['family_profile'].value_counts(dropna = False)

family_profile
married male with children      324135
married female with children    321067
single male, no children        115926
single female, no children      115392
single male with children        23160
single female with children      21590
Name: count, dtype: int64

In [74]:
df_active_customers.head()

,product_id,product_name,aisle_id,department_id,prices,order_id,user_id,order_number,orders_day_of_week,time_of_order,days_since_prior_order,add_to_cart_order,reordered,price_range,price_range_loc,busiest_day,busiest_days,busiest_period_of_day,max_order,loyalty_flag,average_price,spending_flag,median_orders,order_frequency_flag,mean_expenses,gender,state,age,date_joined,n_dependants,fam_status,income,region,activity_flag,age_group,income_group,department,dependants_group,family_profile
0,1,Chocolate Sandwich Cookies,61,19,5.8,3139998,138,28,6,11,3.0,5,0,Mid-range product,Mid-range product,Regularly busy,Regularly busy,Most orders,30,Regular customer,12.900000,High spender,6.5,Frequent customer,5.8,Male,Minnesota,81,2019-08-01,1,married,49620,Midwest,active customer,60-81 years old,Low Income,snacks,Has dependants,married male with children
1,1,Chocolate Sandwich Cookies,61,19,5.8,1977647,138,30,6,17,20.0,1,1,Mid-range product,Mid-range product,Regularly busy,Regularly busy,Average orders,30,Regular customer,12.900000,High spender,6.5,Frequent customer,5.8,Male,Minnesota,81,2019-08-01,1,married,49620,Midwest,active customer,60-81 years old,Low Income,snacks,Has dependants,married male with children
2,1,Chocolate Sandwich Cookies,61,19,5.8,389851,709,2,0,21,6.0,20,0,Mid-range product,Mid-range product,Busiest day,Busiest days,Average orders,5,New customer,5.288889,Low spender,7.0,Frequent customer,5.8,Female,Vermont,66,2018-06-16,2,married,158302,Northeast,active customer,60-81 years old,High Income,snacks,Has dependants,married female with children
3,955,Cheddar Chex Mix,125,19,3.9,147387,709,4,0,21,8.0,30,0,Low-range product,Low-range product,Busiest day,Busiest days,Average orders,5,New customer,5.288889,Low spender,7.0,Frequent customer,3.9,Female,Vermont,66,2018-06-16,2,married,158302,Northeast,active customer,60-81 years old,High Income,snacks,Has dependants,married female with children
4,1,Chocolate Sandwich Cookies,61,19,5.8,1701441,777,16,1,7,26.0,7,0,Mid-range product,Mid-range product,Regularly busy,Busiest days,Average orders,16,Regular customer,5.800000,Low spender,26.0,Non-frequent customer,5.8,Female,Hawaii,51,2019-01-04,2,married,57797,West,active customer,45-59 years old,Low Income,snacks,Has dependants,married female with children


In [75]:
#Exporting
df_active_customers.to_pickle(os.path.join(path, '02 Data', 'Prepared Data', 'df_active_customers_4_10.pkl'))